# Import libraries and data

In [1]:
# Import libraries
import os
import openai
import pandas as pd
import numpy as np
import openpyxl

In [2]:
# Get OpenAI API secret key (change this file with your own OpenAI key)
open_ai_key = open('../data/local/openai_key.txt', 'r').read()
openai.api_key = open_ai_key
#openai.Model.list()

In [3]:
# Prompt for GPT
prompt_raw = "I would like to summarise in maximum 300 characters and in English what the following sentences in Portuguese talk about. \
Do not use aspects mentioned only in one sentence as part of the summary: TWEETS"

# 2017 negative

## January

In [4]:
# Get monthly data
df_2017_01_neg = pd.read_csv('../data/local/df_neg_2017-01.csv').reset_index()
df_2017_01_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2204 entries, 0 to 2203
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   2204 non-null   int64  
 1   LocationCode            2204 non-null   object 
 2   LocationName            2204 non-null   object 
 3   StatesCode              2204 non-null   object 
 4   ...4                    2204 non-null   int64  
 5   id                      2204 non-null   float64
 6   text                    2204 non-null   object 
 7   label                   2204 non-null   object 
 8   in_reply_to_status_id   213 non-null    float64
 9   in_reply_to_user_id     221 non-null    float64
 10  quoted_user_id          55 non-null     float64
 11  quoted_status_id        55 non-null     float64
 12  retweeted_user_id       351 non-null    float64
 13  retweeted_status_id     351 non-null    float64
 14  created_at              2204 non-null   

### Representative sample based on week and location

In [5]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_01_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_01_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w01,Alagoas,1,0.000454
1,2017-w01,Amazonas,2,0.000907
2,2017-w01,Bahia,8,0.003630
3,2017-w01,Ceara,5,0.002269
4,2017-w01,Distrito Federal,11,0.004991
...,...,...,...,...
96,2017-w05,Paraiba,4,0.001815
97,2017-w05,Parana,2,0.000907
98,2017-w05,Rio Grande Do Sul,16,0.007260
99,2017-w05,Santa Catarina,2,0.000907


In [6]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w01,Alagoas,1,0.000454,1
1,2017-w01,Amazonas,2,0.000907,2
2,2017-w01,Bahia,8,0.003630,8
3,2017-w01,Ceara,5,0.002269,5
4,2017-w01,Distrito Federal,11,0.004991,12
...,...,...,...,...,...
96,2017-w05,Paraiba,4,0.001815,4
97,2017-w05,Parana,2,0.000907,2
98,2017-w05,Rio Grande Do Sul,16,0.007260,17
99,2017-w05,Santa Catarina,2,0.000907,2


In [7]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_01_neg_sample_size = pd.merge(df_2017_01_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [10]:
# Step 6: Perform the stratified sampling
#df_2017_01_neg_sample = df_2017_01_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2017_01_neg_sample = df_2017_01_neg
# Step 7: Check the result
print(f"Sampled {len(df_2017_01_neg_sample)} rows.")


Sampled 2204 rows.


In [11]:
text_gpt_2017_01_neg = '\n'.join(df_2017_01_neg_sample['text'].astype(str))

### Prompts and loops

In [12]:
prompt_1 = prompt_raw.replace('TWEETS', text_gpt_2017_01_neg)

In [13]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_1}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [14]:
response.choices[0].message.content

"The passages mainly express frustration and fear associated with vaccines, particularly against the backdrop of a febre amarela outbreak. Many individuals report experiencing pain and discomfort after vaccinations, while others complain about the difficulties in accessing vaccines, such as long lines and shortages. There's also a recurring theme of anxiety about the side effects of vaccines, alongside skepticism towards vaccination campaigns and the healthcare system. Some express outright resistance to getting vaccinated, influenced by personal beliefs or misinformation."

In [15]:
summary = []
dataset = []

In [16]:
dataset.append("2017_01_negitive")

In [17]:
summary.append(response.choices[0].message.content)

In [18]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  1 non-null      object
 1   summary  1 non-null      object
dtypes: object(2)
memory usage: 144.0+ bytes


In [19]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...


In [20]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcaDC5iRa1nZ09pAYGEzy4ipMoHy at 0x254184caa40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The passages mainly express frustration and fear associated with vaccines, particularly against the backdrop of a febre amarela outbreak. Many individuals report experiencing pain and discomfort after vaccinations, while others complain about the difficulties in accessing vaccines, such as long lines and shortages. There's also a recurring theme of anxiety about the side effects of vaccines, alongside skepticism towards vaccination campaigns and the healthcare system. Some express outright resistance to getting vaccinated, influenced by personal beliefs or misinformation.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861517,
  "id": "chatcmpl-AUcaDC5iRa1nZ09pAYGEzy4ipMoHy",
  "model": "gpt-4o-mini-2024-07-18",
  "o

## February

In [21]:
# Get monthly data
df_2017_02_neg = pd.read_csv('../data/local/df_neg_2017-02.csv').reset_index()
df_2017_02_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1700 non-null   int64  
 1   LocationCode            1700 non-null   object 
 2   LocationName            1700 non-null   object 
 3   StatesCode              1700 non-null   object 
 4   ...4                    1700 non-null   int64  
 5   id                      1700 non-null   float64
 6   text                    1700 non-null   object 
 7   label                   1700 non-null   object 
 8   in_reply_to_status_id   136 non-null    float64
 9   in_reply_to_user_id     147 non-null    float64
 10  quoted_user_id          43 non-null     float64
 11  quoted_status_id        43 non-null     float64
 12  retweeted_user_id       134 non-null    float64
 13  retweeted_status_id     134 non-null    float64
 14  created_at              1700 non-null   

### Representative sample based on week and location

In [22]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_02_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_02_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w05,Acre,2,0.001176
1,2017-w05,Alagoas,4,0.002353
2,2017-w05,Amazonas,5,0.002941
3,2017-w05,Bahia,21,0.012353
4,2017-w05,Ceara,1,0.000588
...,...,...,...,...
86,2017-w09,Minas Gerais,13,0.007647
87,2017-w09,Parana,1,0.000588
88,2017-w09,Pernambuco,1,0.000588
89,2017-w09,Rio Grande Do Sul,3,0.001765


In [23]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w05,Acre,2,0.001176,2
1,2017-w05,Alagoas,4,0.002353,5
2,2017-w05,Amazonas,5,0.002941,7
3,2017-w05,Bahia,21,0.012353,30
4,2017-w05,Ceara,1,0.000588,1
...,...,...,...,...,...
86,2017-w09,Minas Gerais,13,0.007647,18
87,2017-w09,Parana,1,0.000588,1
88,2017-w09,Pernambuco,1,0.000588,1
89,2017-w09,Rio Grande Do Sul,3,0.001765,4


In [24]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_02_neg_sample_size = pd.merge(df_2017_02_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [27]:
# Step 6: Perform the stratified sampling
#df_2017_02_neg_sample = df_2017_02_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2017_02_neg_sample = df_2017_02_neg
# Step 7: Check the result
print(f"Sampled {len(df_2017_02_neg_sample)} rows.")


Sampled 1700 rows.


In [28]:
text_gpt_2017_02_neg = '\n'.join(df_2017_02_neg_sample['text'].astype(str))

### Prompts and loops

In [29]:
prompt_2 = prompt_raw.replace('TWEETS', text_gpt_2017_02_neg)

In [30]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_2}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [31]:
response.choices[0].message.content

'The sentences express a common frustration and anxiety surrounding vaccinations, particularly for diseases like tetanus and yellow fever. Many complain about pain in their arms, the long waiting times at clinics, and negative experiences related to receiving vaccines. There are also comments on the fear of needles, the burden of mandatory vaccinations for traveling, and reactions to vaccines that leave individuals feeling unwell. Overall, the sentiment reflects a mix of dread, annoyance, and physical discomfort associated with the vaccination process.'

In [32]:
#summary = []
#dataset = []

In [33]:
dataset.append("2017_02_negitive")

In [34]:
summary.append(response.choices[0].message.content)

In [35]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  2 non-null      object
 1   summary  2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


In [36]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...
1,2017_02_negitive,The sentences express a common frustration and...


In [37]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcaoBtppnYIJuKKmtdMlLpm3Ql2S at 0x25413e867c0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences express a common frustration and anxiety surrounding vaccinations, particularly for diseases like tetanus and yellow fever. Many complain about pain in their arms, the long waiting times at clinics, and negative experiences related to receiving vaccines. There are also comments on the fear of needles, the burden of mandatory vaccinations for traveling, and reactions to vaccines that leave individuals feeling unwell. Overall, the sentiment reflects a mix of dread, annoyance, and physical discomfort associated with the vaccination process.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861554,
  "id": "chatcmpl-AUcaoBtppnYIJuKKmtdMlLpm3Ql2S",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.complet

## March

In [38]:
# Get monthly data
df_2017_03_neg = pd.read_csv('../data/local/df_neg_2017-03.csv').reset_index()
df_2017_03_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4738 entries, 0 to 4737
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   4738 non-null   int64  
 1   LocationCode            4738 non-null   object 
 2   LocationName            4738 non-null   object 
 3   StatesCode              4738 non-null   object 
 4   ...4                    4738 non-null   int64  
 5   id                      4738 non-null   float64
 6   text                    4738 non-null   object 
 7   label                   4738 non-null   object 
 8   in_reply_to_status_id   310 non-null    float64
 9   in_reply_to_user_id     332 non-null    float64
 10  quoted_user_id          96 non-null     float64
 11  quoted_status_id        96 non-null     float64
 12  retweeted_user_id       654 non-null    float64
 13  retweeted_status_id     654 non-null    float64
 14  created_at              4738 non-null   

### Representative sample based on week and location

In [39]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_03_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_03_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w09,Acre,1,0.000211
1,2017-w09,Alagoas,3,0.000633
2,2017-w09,Amapa,1,0.000211
3,2017-w09,Amazonas,2,0.000422
4,2017-w09,Bahia,12,0.002533
...,...,...,...,...
108,2017-w13,Rondonia,1,0.000211
109,2017-w13,Roraima,1,0.000211
110,2017-w13,Santa Catarina,10,0.002111
111,2017-w13,Sao Paulo,59,0.012453


In [40]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w09,Acre,1,0.000211,0
1,2017-w09,Alagoas,3,0.000633,1
2,2017-w09,Amapa,1,0.000211,0
3,2017-w09,Amazonas,2,0.000422,1
4,2017-w09,Bahia,12,0.002533,6
...,...,...,...,...,...
108,2017-w13,Rondonia,1,0.000211,0
109,2017-w13,Roraima,1,0.000211,0
110,2017-w13,Santa Catarina,10,0.002111,5
111,2017-w13,Sao Paulo,59,0.012453,30


In [41]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_03_neg_sample_size = pd.merge(df_2017_03_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [42]:
# Step 6: Perform the stratified sampling
df_2017_03_neg_sample = df_2017_03_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
).reset_index(drop=True)

# Step 7: Check the result
print(f"Sampled {len(df_2017_03_neg_sample)} rows.")


Sampled 2401 rows.


In [43]:
text_gpt_2017_03_neg = '\n'.join(df_2017_03_neg_sample['text'].astype(str))

### Prompts and loops

In [44]:
prompt_3 = prompt_raw.replace('TWEETS', text_gpt_2017_03_neg)

In [45]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_3}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [46]:
response.choices[0].message.content

'The texts express widespread anxiety, frustration, and humor regarding vaccinations, particularly against yellow fever. Many individuals share their experiences of preparing to receive vaccines, often highlighting long wait times, pain, and fears of side effects. There are also criticisms of government inefficiencies, misinformation about vaccines, and personal anecdotes of reluctance to get vaccinated, reflecting broader societal concerns about health and safety.'

In [47]:
#summary = []
#dataset = []

In [48]:
dataset.append("2017_03_negitive")

In [49]:
summary.append(response.choices[0].message.content)

In [50]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  3 non-null      object
 1   summary  3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes


In [51]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...
1,2017_02_negitive,The sentences express a common frustration and...
2,2017_03_negitive,"The texts express widespread anxiety, frustrat..."


In [52]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcavgKHG7RBPcn3jYDOTHj1hJY1h at 0x254189fa720> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The texts express widespread anxiety, frustration, and humor regarding vaccinations, particularly against yellow fever. Many individuals share their experiences of preparing to receive vaccines, often highlighting long wait times, pain, and fears of side effects. There are also criticisms of government inefficiencies, misinformation about vaccines, and personal anecdotes of reluctance to get vaccinated, reflecting broader societal concerns about health and safety.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861561,
  "id": "chatcmpl-AUcavgKHG7RBPcn3jYDOTHj1hJY1h",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_9b78b61c52",
  "usage": {
    "completion_tokens": 75,


## April

In [53]:
# Get monthly data
df_2017_04_neg = pd.read_csv('../data/local/df_neg_2017-04.csv').reset_index()
df_2017_04_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2701 entries, 0 to 2700
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   2701 non-null   int64  
 1   LocationCode            2701 non-null   object 
 2   LocationName            2701 non-null   object 
 3   StatesCode              2701 non-null   object 
 4   ...4                    2701 non-null   int64  
 5   id                      2701 non-null   float64
 6   text                    2701 non-null   object 
 7   label                   2701 non-null   object 
 8   in_reply_to_status_id   184 non-null    float64
 9   in_reply_to_user_id     196 non-null    float64
 10  quoted_user_id          57 non-null     float64
 11  quoted_status_id        57 non-null     float64
 12  retweeted_user_id       258 non-null    float64
 13  retweeted_status_id     258 non-null    float64
 14  created_at              2701 non-null   

### Representative sample based on week and location

In [54]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_04_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_04_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w13,Alagoas,1,0.000370
1,2017-w13,Amapa,1,0.000370
2,2017-w13,Bahia,6,0.002221
3,2017-w13,Espirito Santo,9,0.003332
4,2017-w13,Goias,2,0.000740
...,...,...,...,...
107,2017-w18,Paraiba,1,0.000370
108,2017-w18,Parana,3,0.001111
109,2017-w18,Rio Grande Do Sul,3,0.001111
110,2017-w18,Rondonia,1,0.000370


In [55]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w13,Alagoas,1,0.000370,0
1,2017-w13,Amapa,1,0.000370,0
2,2017-w13,Bahia,6,0.002221,5
3,2017-w13,Espirito Santo,9,0.003332,8
4,2017-w13,Goias,2,0.000740,1
...,...,...,...,...,...
107,2017-w18,Paraiba,1,0.000370,0
108,2017-w18,Parana,3,0.001111,2
109,2017-w18,Rio Grande Do Sul,3,0.001111,2
110,2017-w18,Rondonia,1,0.000370,0


In [56]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_04_neg_sample_size = pd.merge(df_2017_04_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [57]:
# Step 6: Perform the stratified sampling
df_2017_04_neg_sample = df_2017_04_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
).reset_index(drop=True)

# Step 7: Check the result
print(f"Sampled {len(df_2017_04_neg_sample)} rows.")


Sampled 2380 rows.


In [58]:
text_gpt_2017_04_neg = '\n'.join(df_2017_04_neg_sample['text'].astype(str))

### Prompts and loops

In [59]:
prompt_4 = prompt_raw.replace('TWEETS', text_gpt_2017_04_neg)

In [60]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_4}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [61]:
response.choices[0].message.content

'The texts reflect widespread anxiety and negative sentiments around vaccinations, particularly the yellow fever vaccine. Many express fear of needles, concern about adverse reactions, and frustration with the vaccination process, including long wait times and lack of availability. Some mention specific incidents of adverse reactions or deaths linked to vaccines, contributing to skepticism. Overall, there is a strong emotional response to the idea of vaccination, ranging from dread to outright refusal.'

In [62]:
#summary = []
#dataset = []

In [63]:
dataset.append("2017_04_negitive")

In [64]:
summary.append(response.choices[0].message.content)

In [65]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  4 non-null      object
 1   summary  4 non-null      object
dtypes: object(2)
memory usage: 192.0+ bytes


In [66]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...
1,2017_02_negitive,The sentences express a common frustration and...
2,2017_03_negitive,"The texts express widespread anxiety, frustrat..."
3,2017_04_negitive,The texts reflect widespread anxiety and negat...


In [67]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcb37DPqh3YsNqUPYEWvShxt1Xn4 at 0x25418a182c0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The texts reflect widespread anxiety and negative sentiments around vaccinations, particularly the yellow fever vaccine. Many express fear of needles, concern about adverse reactions, and frustration with the vaccination process, including long wait times and lack of availability. Some mention specific incidents of adverse reactions or deaths linked to vaccines, contributing to skepticism. Overall, there is a strong emotional response to the idea of vaccination, ranging from dread to outright refusal.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861569,
  "id": "chatcmpl-AUcb37DPqh3YsNqUPYEWvShxt1Xn4",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "u

## May

In [68]:
# Get monthly data
df_2017_05_neg = pd.read_csv('../data/local/df_neg_2017-05.csv').reset_index()
df_2017_05_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4381 entries, 0 to 4380
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   4381 non-null   int64  
 1   LocationCode            4381 non-null   object 
 2   LocationName            4381 non-null   object 
 3   StatesCode              4381 non-null   object 
 4   ...4                    4381 non-null   int64  
 5   id                      4381 non-null   float64
 6   text                    4381 non-null   object 
 7   label                   4381 non-null   object 
 8   in_reply_to_status_id   283 non-null    float64
 9   in_reply_to_user_id     297 non-null    float64
 10  quoted_user_id          101 non-null    float64
 11  quoted_status_id        101 non-null    float64
 12  retweeted_user_id       1022 non-null   float64
 13  retweeted_status_id     1022 non-null   float64
 14  created_at              4381 non-null   

### Representative sample based on week and location

In [69]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_05_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_05_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w18,Acre,1,0.000228
1,2017-w18,Alagoas,1,0.000228
2,2017-w18,Amapa,1,0.000228
3,2017-w18,Amazonas,6,0.001370
4,2017-w18,Bahia,17,0.003880
...,...,...,...,...
110,2017-w22,Rondonia,1,0.000228
111,2017-w22,Santa Catarina,11,0.002511
112,2017-w22,Sao Paulo,50,0.011413
113,2017-w22,Sergipe,1,0.000228


In [70]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w18,Acre,1,0.000228,0
1,2017-w18,Alagoas,1,0.000228,0
2,2017-w18,Amapa,1,0.000228,0
3,2017-w18,Amazonas,6,0.001370,3
4,2017-w18,Bahia,17,0.003880,9
...,...,...,...,...,...
110,2017-w22,Rondonia,1,0.000228,0
111,2017-w22,Santa Catarina,11,0.002511,6
112,2017-w22,Sao Paulo,50,0.011413,27
113,2017-w22,Sergipe,1,0.000228,0


In [71]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_05_neg_sample_size = pd.merge(df_2017_05_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [72]:
# Step 6: Perform the stratified sampling
df_2017_05_neg_sample = df_2017_05_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
).reset_index(drop=True)

# Step 7: Check the result
print(f"Sampled {len(df_2017_05_neg_sample)} rows.")


Sampled 2399 rows.


In [73]:
text_gpt_2017_05_neg = '\n'.join(df_2017_05_neg_sample['text'].astype(str))

### Prompts and loops

In [74]:
prompt_5 = prompt_raw.replace('TWEETS', text_gpt_2017_05_neg)

In [75]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_5}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [76]:
response.choices[0].message.content

"The sentences express a strong dislike and fear of vaccinations, particularly the flu shot. Many individuals report experiencing pain and discomfort in their arms after receiving the vaccine, often feeling ill or expressing concern about potential side effects. Some mention instances of being forced to get vaccinated by family members, while others express skepticism about the efficacy of vaccines, linking them to negative health outcomes. There is also mention of anti-vaccine sentiments and a growing movement against vaccinations, which raises public health concerns. Overall, there's a mix of humor, frustration, and anxiety surrounding the topic of vaccinations."

In [77]:
#summary = []
#dataset = []

In [78]:
dataset.append("2017_05_negitive")

In [79]:
summary.append(response.choices[0].message.content)

In [80]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  5 non-null      object
 1   summary  5 non-null      object
dtypes: object(2)
memory usage: 208.0+ bytes


In [81]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...
1,2017_02_negitive,The sentences express a common frustration and...
2,2017_03_negitive,"The texts express widespread anxiety, frustrat..."
3,2017_04_negitive,The texts reflect widespread anxiety and negat...
4,2017_05_negitive,The sentences express a strong dislike and fea...


In [82]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcbADoT9d98ROXQS6AjWn8ys1PJ8 at 0x25418b5f4f0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences express a strong dislike and fear of vaccinations, particularly the flu shot. Many individuals report experiencing pain and discomfort in their arms after receiving the vaccine, often feeling ill or expressing concern about potential side effects. Some mention instances of being forced to get vaccinated by family members, while others express skepticism about the efficacy of vaccines, linking them to negative health outcomes. There is also mention of anti-vaccine sentiments and a growing movement against vaccinations, which raises public health concerns. Overall, there's a mix of humor, frustration, and anxiety surrounding the topic of vaccinations.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861576

## June

In [83]:
# Get monthly data
df_2017_06_neg = pd.read_csv('../data/local/df_neg_2017-06.csv').reset_index()
df_2017_06_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3254 entries, 0 to 3253
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   3254 non-null   int64  
 1   LocationCode            3254 non-null   object 
 2   LocationName            3254 non-null   object 
 3   StatesCode              3254 non-null   object 
 4   ...4                    3254 non-null   int64  
 5   id                      3254 non-null   float64
 6   text                    3254 non-null   object 
 7   label                   3254 non-null   object 
 8   in_reply_to_status_id   286 non-null    float64
 9   in_reply_to_user_id     295 non-null    float64
 10  quoted_user_id          76 non-null     float64
 11  quoted_status_id        76 non-null     float64
 12  retweeted_user_id       534 non-null    float64
 13  retweeted_status_id     534 non-null    float64
 14  created_at              3254 non-null   

### Representative sample based on week and location

In [84]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_06_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_06_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w22,Acre,1,0.000307
1,2017-w22,Alagoas,3,0.000922
2,2017-w22,Amapa,3,0.000922
3,2017-w22,Amazonas,7,0.002151
4,2017-w22,Bahia,11,0.003380
...,...,...,...,...
110,2017-w26,Rio Grande Do Sul,29,0.008912
111,2017-w26,Roraima,1,0.000307
112,2017-w26,Santa Catarina,6,0.001844
113,2017-w26,Sao Paulo,69,0.021205


In [85]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w22,Acre,1,0.000307,0
1,2017-w22,Alagoas,3,0.000922,2
2,2017-w22,Amapa,3,0.000922,2
3,2017-w22,Amazonas,7,0.002151,5
4,2017-w22,Bahia,11,0.003380,8
...,...,...,...,...,...
110,2017-w26,Rio Grande Do Sul,29,0.008912,21
111,2017-w26,Roraima,1,0.000307,0
112,2017-w26,Santa Catarina,6,0.001844,4
113,2017-w26,Sao Paulo,69,0.021205,51


In [86]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_06_neg_sample_size = pd.merge(df_2017_06_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [87]:
# Step 6: Perform the stratified sampling
df_2017_06_neg_sample = df_2017_06_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
).reset_index(drop=True)

# Step 7: Check the result
print(f"Sampled {len(df_2017_06_neg_sample)} rows.")


Sampled 2396 rows.


In [88]:
text_gpt_2017_06_neg = '\n'.join(df_2017_06_neg_sample['text'].astype(str))

### Prompts and loops

In [89]:
prompt_6 = prompt_raw.replace('TWEETS', text_gpt_2017_06_neg)

In [90]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_6}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [91]:
response.choices[0].message.content

'The texts discuss various aspects of vaccination, particularly against flu and diseases like measles and hepatitis. They highlight public health campaigns, reactions to vaccines (including pain and illness), and concerns about vaccine safety and efficacy. There are also mentions of anti-vaccine sentiments and individual experiences with vaccination, including fear and skepticism. Overall, the discourse reflects a mix of support for vaccination and criticism of its perceived side effects and safety.'

In [92]:
#summary = []
#dataset = []

In [93]:
dataset.append("2017_06_negitive")

In [94]:
summary.append(response.choices[0].message.content)

In [95]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  6 non-null      object
 1   summary  6 non-null      object
dtypes: object(2)
memory usage: 224.0+ bytes


In [96]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...
1,2017_02_negitive,The sentences express a common frustration and...
2,2017_03_negitive,"The texts express widespread anxiety, frustrat..."
3,2017_04_negitive,The texts reflect widespread anxiety and negat...
4,2017_05_negitive,The sentences express a strong dislike and fea...
5,2017_06_negitive,The texts discuss various aspects of vaccinati...


In [97]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcbIvsj2ePcolMhGXEj10WdkXrCM at 0x2541933b770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The texts discuss various aspects of vaccination, particularly against flu and diseases like measles and hepatitis. They highlight public health campaigns, reactions to vaccines (including pain and illness), and concerns about vaccine safety and efficacy. There are also mentions of anti-vaccine sentiments and individual experiences with vaccination, including fear and skepticism. Overall, the discourse reflects a mix of support for vaccination and criticism of its perceived side effects and safety.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861584,
  "id": "chatcmpl-AUcbIvsj2ePcolMhGXEj10WdkXrCM",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usag

## July

In [98]:
# Get monthly data
df_2017_07_neg = pd.read_csv('../data/local/df_neg_2017-07.csv').reset_index()
df_2017_07_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1556 entries, 0 to 1555
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1556 non-null   int64  
 1   LocationCode            1556 non-null   object 
 2   LocationName            1556 non-null   object 
 3   StatesCode              1556 non-null   object 
 4   ...4                    1556 non-null   int64  
 5   id                      1556 non-null   float64
 6   text                    1556 non-null   object 
 7   label                   1556 non-null   object 
 8   in_reply_to_status_id   152 non-null    float64
 9   in_reply_to_user_id     159 non-null    float64
 10  quoted_user_id          76 non-null     float64
 11  quoted_status_id        76 non-null     float64
 12  retweeted_user_id       257 non-null    float64
 13  retweeted_status_id     257 non-null    float64
 14  created_at              1556 non-null   

### Representative sample based on week and location

In [99]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_07_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_07_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w26,Acre,1,0.000643
1,2017-w26,Bahia,4,0.002571
2,2017-w26,Ceara,1,0.000643
3,2017-w26,Distrito Federal,2,0.001285
4,2017-w26,Espirito Santo,2,0.001285
...,...,...,...,...
105,2017-w31,Parana,2,0.001285
106,2017-w31,Pernambuco,2,0.001285
107,2017-w31,Rio Grande Do Sul,6,0.003856
108,2017-w31,Santa Catarina,1,0.000643


In [100]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w26,Acre,1,0.000643,1
1,2017-w26,Bahia,4,0.002571,6
2,2017-w26,Ceara,1,0.000643,1
3,2017-w26,Distrito Federal,2,0.001285,3
4,2017-w26,Espirito Santo,2,0.001285,3
...,...,...,...,...,...
105,2017-w31,Parana,2,0.001285,3
106,2017-w31,Pernambuco,2,0.001285,3
107,2017-w31,Rio Grande Do Sul,6,0.003856,9
108,2017-w31,Santa Catarina,1,0.000643,1


In [101]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_07_neg_sample_size = pd.merge(df_2017_07_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [104]:
# Step 6: Perform the stratified sampling
#df_2017_07_neg_sample = df_2017_07_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2017_07_neg_sample = df_2017_07_neg
# Step 7: Check the result
print(f"Sampled {len(df_2017_07_neg_sample)} rows.")


Sampled 1556 rows.


In [105]:
text_gpt_2017_07_neg = '\n'.join(df_2017_07_neg_sample['text'].astype(str))

### Prompts and loops

In [106]:
prompt_7 = prompt_raw.replace('TWEETS', text_gpt_2017_07_neg)

In [107]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_7}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [108]:
response.choices[0].message.content

'The text discusses various challenges regarding vaccination, particularly the HPV vaccine in Brazil. It highlights prejudice, misinformation, and low demand for vaccines, which worry health authorities. There are also mentions of personal experiences with vaccine reactions, criticisms of anti-vaccine movements, and the need for public awareness.'

In [109]:
#summary = []
#dataset = []

In [110]:
dataset.append("2017_07_negitive")

In [111]:
summary.append(response.choices[0].message.content)

In [112]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  7 non-null      object
 1   summary  7 non-null      object
dtypes: object(2)
memory usage: 240.0+ bytes


In [113]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...
1,2017_02_negitive,The sentences express a common frustration and...
2,2017_03_negitive,"The texts express widespread anxiety, frustrat..."
3,2017_04_negitive,The texts reflect widespread anxiety and negat...
4,2017_05_negitive,The sentences express a strong dislike and fea...
5,2017_06_negitive,The texts discuss various aspects of vaccinati...
6,2017_07_negitive,The text discusses various challenges regardin...


In [114]:
response

<OpenAIObject chat.completion id=chatcmpl-AUccHkRKGspUpRZLdTrnHwe1DwQXR at 0x254195bd9a0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various challenges regarding vaccination, particularly the HPV vaccine in Brazil. It highlights prejudice, misinformation, and low demand for vaccines, which worry health authorities. There are also mentions of personal experiences with vaccine reactions, criticisms of anti-vaccine movements, and the need for public awareness.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861645,
  "id": "chatcmpl-AUccHkRKGspUpRZLdTrnHwe1DwQXR",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 57,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_token

## August

In [115]:
# Get monthly data
df_2017_08_neg = pd.read_csv('../data/local/df_neg_2017-08.csv').reset_index()
df_2017_08_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4108 entries, 0 to 4107
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   4108 non-null   int64  
 1   LocationCode            4108 non-null   object 
 2   LocationName            4108 non-null   object 
 3   StatesCode              4108 non-null   object 
 4   ...4                    4108 non-null   int64  
 5   id                      4108 non-null   float64
 6   text                    4108 non-null   object 
 7   label                   4108 non-null   object 
 8   in_reply_to_status_id   184 non-null    float64
 9   in_reply_to_user_id     193 non-null    float64
 10  quoted_user_id          40 non-null     float64
 11  quoted_status_id        40 non-null     float64
 12  retweeted_user_id       2483 non-null   float64
 13  retweeted_status_id     2483 non-null   float64
 14  created_at              4108 non-null   

### Representative sample based on week and location

In [116]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_08_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_08_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w31,Alagoas,2,0.000487
1,2017-w31,Amazonas,2,0.000487
2,2017-w31,Bahia,4,0.000974
3,2017-w31,Ceara,1,0.000243
4,2017-w31,Distrito Federal,6,0.001461
...,...,...,...,...
108,2017-w35,Rio Grande Do Sul,92,0.022395
109,2017-w35,Rondonia,1,0.000243
110,2017-w35,Santa Catarina,4,0.000974
111,2017-w35,Sao Paulo,61,0.014849


In [117]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w31,Alagoas,2,0.000487,1
1,2017-w31,Amazonas,2,0.000487,1
2,2017-w31,Bahia,4,0.000974,2
3,2017-w31,Ceara,1,0.000243,0
4,2017-w31,Distrito Federal,6,0.001461,3
...,...,...,...,...,...
108,2017-w35,Rio Grande Do Sul,92,0.022395,54
109,2017-w35,Rondonia,1,0.000243,0
110,2017-w35,Santa Catarina,4,0.000974,2
111,2017-w35,Sao Paulo,61,0.014849,36


In [118]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_08_neg_sample_size = pd.merge(df_2017_08_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [119]:
# Step 6: Perform the stratified sampling
df_2017_08_neg_sample = df_2017_08_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
).reset_index(drop=True)

# Step 7: Check the result
print(f"Sampled {len(df_2017_08_neg_sample)} rows.")


Sampled 2387 rows.


In [120]:
text_gpt_2017_08_neg = '\n'.join(df_2017_08_neg_sample['text'].astype(str))

### Prompts and loops

In [121]:
prompt_8 = prompt_raw.replace('TWEETS', text_gpt_2017_08_neg)

In [122]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_8}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [123]:
response.choices[0].message.content

'The text discusses various opinions and experiences regarding vaccinations, particularly focusing on pain and discomfort associated with receiving them. It touches on the stigma and hesitance surrounding vaccines, including the HPV vaccine, and mentions a rise in anti-vaccination sentiments. There are also references to the importance of vaccination in preventing diseases and public health concerns related to declining vaccination rates.'

In [124]:
#summary = []
#dataset = []

In [125]:
dataset.append("2017_08_negitive")

In [126]:
summary.append(response.choices[0].message.content)

In [127]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  8 non-null      object
 1   summary  8 non-null      object
dtypes: object(2)
memory usage: 256.0+ bytes


In [128]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...
1,2017_02_negitive,The sentences express a common frustration and...
2,2017_03_negitive,"The texts express widespread anxiety, frustrat..."
3,2017_04_negitive,The texts reflect widespread anxiety and negat...
4,2017_05_negitive,The sentences express a strong dislike and fea...
5,2017_06_negitive,The texts discuss various aspects of vaccinati...
6,2017_07_negitive,The text discusses various challenges regardin...
7,2017_08_negitive,The text discusses various opinions and experi...


In [129]:
response

<OpenAIObject chat.completion id=chatcmpl-AUccNgXKKXX9sArVzM2z44QohZRTp at 0x254195c5cc0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various opinions and experiences regarding vaccinations, particularly focusing on pain and discomfort associated with receiving them. It touches on the stigma and hesitance surrounding vaccines, including the HPV vaccine, and mentions a rise in anti-vaccination sentiments. There are also references to the importance of vaccination in preventing diseases and public health concerns related to declining vaccination rates.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861651,
  "id": "chatcmpl-AUccNgXKKXX9sArVzM2z44QohZRTp",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 71,
    "completion_tokens_deta

## September

In [130]:
# Get monthly data
df_2017_09_neg = pd.read_csv('../data/local/df_neg_2017-09.csv').reset_index()
df_2017_09_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2033 entries, 0 to 2032
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   2033 non-null   int64  
 1   LocationCode            2033 non-null   object 
 2   LocationName            2033 non-null   object 
 3   StatesCode              2033 non-null   object 
 4   ...4                    2033 non-null   int64  
 5   id                      2033 non-null   float64
 6   text                    2033 non-null   object 
 7   label                   2033 non-null   object 
 8   in_reply_to_status_id   229 non-null    float64
 9   in_reply_to_user_id     233 non-null    float64
 10  quoted_user_id          54 non-null     float64
 11  quoted_status_id        54 non-null     float64
 12  retweeted_user_id       430 non-null    float64
 13  retweeted_status_id     430 non-null    float64
 14  created_at              2033 non-null   

### Representative sample based on week and location

In [131]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_09_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_09_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w35,Bahia,3,0.001476
1,2017-w35,Ceara,3,0.001476
2,2017-w35,Distrito Federal,2,0.000984
3,2017-w35,Espirito Santo,8,0.003935
4,2017-w35,Goias,5,0.002459
...,...,...,...,...
101,2017-w39,Rio De Janeiro,2,0.000984
102,2017-w39,Rio Grande Do Sul,32,0.015740
103,2017-w39,Rondonia,1,0.000492
104,2017-w39,Santa Catarina,8,0.003935


In [132]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w35,Bahia,3,0.001476,3
1,2017-w35,Ceara,3,0.001476,3
2,2017-w35,Distrito Federal,2,0.000984,2
3,2017-w35,Espirito Santo,8,0.003935,9
4,2017-w35,Goias,5,0.002459,6
...,...,...,...,...,...
101,2017-w39,Rio De Janeiro,2,0.000984,2
102,2017-w39,Rio Grande Do Sul,32,0.015740,38
103,2017-w39,Rondonia,1,0.000492,1
104,2017-w39,Santa Catarina,8,0.003935,9


In [133]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_09_neg_sample_size = pd.merge(df_2017_09_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [135]:
# Step 6: Perform the stratified sampling
#df_2017_09_neg_sample = df_2017_09_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2017_09_neg_sample = df_2017_09_neg
# Step 7: Check the result
print(f"Sampled {len(df_2017_09_neg_sample)} rows.")


Sampled 2033 rows.


In [136]:
text_gpt_2017_09_neg = '\n'.join(df_2017_09_neg_sample['text'].astype(str))

### Prompts and loops

In [137]:
prompt_9 = prompt_raw.replace('TWEETS', text_gpt_2017_09_neg)

In [138]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_9}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [139]:
response.choices[0].message.content

'The text reflects a societal discourse around vaccination, highlighting a mix of concerns, fears, and opinions. It mentions the historical context of scientific advancement in 1956 versus contemporary skepticism and conspiracy theories from 2017, including anti-vaccine movements, the belief in a flat Earth, and climate change denial. There are personal anecdotes about the pain and discomfort associated with vaccinations, alongside criticism of misinformation surrounding vaccines and their perceived dangers. The narrative conveys a sense of frustration regarding public health measures and the rise of irrational beliefs in society.'

In [140]:
#summary = []
#dataset = []

In [141]:
dataset.append("2017_09_negitive")

In [142]:
summary.append(response.choices[0].message.content)

In [143]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  9 non-null      object
 1   summary  9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


In [144]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...
1,2017_02_negitive,The sentences express a common frustration and...
2,2017_03_negitive,"The texts express widespread anxiety, frustrat..."
3,2017_04_negitive,The texts reflect widespread anxiety and negat...
4,2017_05_negitive,The sentences express a strong dislike and fea...
5,2017_06_negitive,The texts discuss various aspects of vaccinati...
6,2017_07_negitive,The text discusses various challenges regardin...
7,2017_08_negitive,The text discusses various opinions and experi...
8,2017_09_negitive,The text reflects a societal discourse around ...


In [145]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcdTYMybeSVlNJ3zFjoFDI8aSDJT at 0x254198bcf40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text reflects a societal discourse around vaccination, highlighting a mix of concerns, fears, and opinions. It mentions the historical context of scientific advancement in 1956 versus contemporary skepticism and conspiracy theories from 2017, including anti-vaccine movements, the belief in a flat Earth, and climate change denial. There are personal anecdotes about the pain and discomfort associated with vaccinations, alongside criticism of misinformation surrounding vaccines and their perceived dangers. The narrative conveys a sense of frustration regarding public health measures and the rise of irrational beliefs in society.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861719,
  "id": "chatcmpl-AUcdTYMybeSVlN

## October

In [146]:
# Get monthly data
df_2017_10_neg = pd.read_csv('../data/local/df_neg_2017-10.csv').reset_index()
df_2017_10_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1512 non-null   int64  
 1   LocationCode            1512 non-null   object 
 2   LocationName            1512 non-null   object 
 3   StatesCode              1512 non-null   object 
 4   ...4                    1512 non-null   int64  
 5   id                      1512 non-null   float64
 6   text                    1512 non-null   object 
 7   label                   1512 non-null   object 
 8   in_reply_to_status_id   168 non-null    float64
 9   in_reply_to_user_id     169 non-null    float64
 10  quoted_user_id          45 non-null     float64
 11  quoted_status_id        45 non-null     float64
 12  retweeted_user_id       229 non-null    float64
 13  retweeted_status_id     229 non-null    float64
 14  created_at              1512 non-null   

### Representative sample based on week and location

In [147]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_10_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_10_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w40,Alagoas,2,0.001323
1,2017-w40,Amapa,1,0.000661
2,2017-w40,Amazonas,3,0.001984
3,2017-w40,Bahia,11,0.007275
4,2017-w40,Ceara,2,0.001323
...,...,...,...,...
96,2017-w44,Rio Grande Do Sul,21,0.013889
97,2017-w44,Rondonia,1,0.000661
98,2017-w44,Roraima,1,0.000661
99,2017-w44,Santa Catarina,1,0.000661


In [148]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w40,Alagoas,2,0.001323,3
1,2017-w40,Amapa,1,0.000661,1
2,2017-w40,Amazonas,3,0.001984,4
3,2017-w40,Bahia,11,0.007275,17
4,2017-w40,Ceara,2,0.001323,3
...,...,...,...,...,...
96,2017-w44,Rio Grande Do Sul,21,0.013889,34
97,2017-w44,Rondonia,1,0.000661,1
98,2017-w44,Roraima,1,0.000661,1
99,2017-w44,Santa Catarina,1,0.000661,1


In [149]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_10_neg_sample_size = pd.merge(df_2017_10_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [151]:
# Step 6: Perform the stratified sampling
#df_2017_10_neg_sample = df_2017_10_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2017_10_neg_sample = df_2017_10_neg
# Step 7: Check the result
print(f"Sampled {len(df_2017_10_neg_sample)} rows.")


Sampled 1512 rows.


In [152]:
text_gpt_2017_10_neg = '\n'.join(df_2017_10_neg_sample['text'].astype(str))

### Prompts and loops

In [153]:
prompt_10 = prompt_raw.replace('TWEETS', text_gpt_2017_10_neg)

In [154]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_10}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [155]:
response.choices[0].message.content

'The sentences discuss a range of opinions and experiences regarding vaccinations, particularly focusing on the pain associated with receiving them, the fear of needles, and personal anecdotes about getting vaccinated. There are also mentions of the consequences of refusing vaccinations, including legal repercussions for parents who do not vaccinate their children. Additionally, some express skepticism about vaccine efficacy, citing the rise of anti-vaccine sentiments and public health concerns related to unvaccinated populations.'

In [156]:
#summary = []
#dataset = []

In [157]:
dataset.append("2017_10_negitive")

In [158]:
summary.append(response.choices[0].message.content)

In [159]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  10 non-null     object
 1   summary  10 non-null     object
dtypes: object(2)
memory usage: 288.0+ bytes


In [160]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...
1,2017_02_negitive,The sentences express a common frustration and...
2,2017_03_negitive,"The texts express widespread anxiety, frustrat..."
3,2017_04_negitive,The texts reflect widespread anxiety and negat...
4,2017_05_negitive,The sentences express a strong dislike and fea...
5,2017_06_negitive,The texts discuss various aspects of vaccinati...
6,2017_07_negitive,The text discusses various challenges regardin...
7,2017_08_negitive,The text discusses various opinions and experi...
8,2017_09_negitive,The text reflects a societal discourse around ...
9,2017_10_negitive,The sentences discuss a range of opinions and ...


In [161]:
response

<OpenAIObject chat.completion id=chatcmpl-AUceF5YJJBZ7qj2lnm66Xc4qOWaxk at 0x254196fea40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences discuss a range of opinions and experiences regarding vaccinations, particularly focusing on the pain associated with receiving them, the fear of needles, and personal anecdotes about getting vaccinated. There are also mentions of the consequences of refusing vaccinations, including legal repercussions for parents who do not vaccinate their children. Additionally, some express skepticism about vaccine efficacy, citing the rise of anti-vaccine sentiments and public health concerns related to unvaccinated populations.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861767,
  "id": "chatcmpl-AUceF5YJJBZ7qj2lnm66Xc4qOWaxk",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_finger

## November

In [162]:
# Get monthly data
df_2017_11_neg = pd.read_csv('../data/local/df_neg_2017-11.csv').reset_index()
df_2017_11_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348 entries, 0 to 1347
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1348 non-null   int64  
 1   LocationCode            1348 non-null   object 
 2   LocationName            1348 non-null   object 
 3   StatesCode              1348 non-null   object 
 4   ...4                    1348 non-null   int64  
 5   id                      1348 non-null   float64
 6   text                    1348 non-null   object 
 7   label                   1348 non-null   object 
 8   in_reply_to_status_id   145 non-null    float64
 9   in_reply_to_user_id     152 non-null    float64
 10  quoted_user_id          39 non-null     float64
 11  quoted_status_id        39 non-null     float64
 12  retweeted_user_id       160 non-null    float64
 13  retweeted_status_id     160 non-null    float64
 14  created_at              1348 non-null   

### Representative sample based on week and location

In [163]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_11_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_11_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w44,Amazonas,1,0.000742
1,2017-w44,Bahia,2,0.001484
2,2017-w44,Ceara,3,0.002226
3,2017-w44,Distrito Federal,3,0.002226
4,2017-w44,Espirito Santo,6,0.004451
...,...,...,...,...
90,2017-w48,Rio Grande Do Sul,27,0.020030
91,2017-w48,Santa Catarina,5,0.003709
92,2017-w48,Sao Paulo,61,0.045252
93,2017-w48,Sergipe,2,0.001484


In [164]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w44,Amazonas,1,0.000742,1
1,2017-w44,Bahia,2,0.001484,3
2,2017-w44,Ceara,3,0.002226,5
3,2017-w44,Distrito Federal,3,0.002226,5
4,2017-w44,Espirito Santo,6,0.004451,10
...,...,...,...,...,...
90,2017-w48,Rio Grande Do Sul,27,0.020030,49
91,2017-w48,Santa Catarina,5,0.003709,9
92,2017-w48,Sao Paulo,61,0.045252,110
93,2017-w48,Sergipe,2,0.001484,3


In [165]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_11_neg_sample_size = pd.merge(df_2017_11_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [167]:
# Step 6: Perform the stratified sampling
#df_2017_11_neg_sample = df_2017_11_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2017_11_neg_sample = df_2017_11_neg
# Step 7: Check the result
print(f"Sampled {len(df_2017_11_neg_sample)} rows.")


Sampled 1348 rows.


In [168]:
text_gpt_2017_11_neg = '\n'.join(df_2017_11_neg_sample['text'].astype(str))

### Prompts and loops

In [169]:
prompt_11 = prompt_raw.replace('TWEETS', text_gpt_2017_11_neg)

In [170]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_11}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [171]:
response.choices[0].message.content

'The text discusses various frustrations and fears related to vaccines, particularly highlighting experiences of pain and anxiety during vaccinations. It mentions the controversy surrounding the dengue vaccine, emphasizing that it should not be administered to individuals who have never had dengue due to potential risks. Concerns about misinformation and vaccine hesitancy are also present, alongside personal anecdotes about the emotional impact of vaccinations on both adults and children.'

In [172]:
#summary = []
#dataset = []

In [173]:
dataset.append("2017_11_negitive")

In [174]:
summary.append(response.choices[0].message.content)

In [175]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  11 non-null     object
 1   summary  11 non-null     object
dtypes: object(2)
memory usage: 304.0+ bytes


In [176]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...
1,2017_02_negitive,The sentences express a common frustration and...
2,2017_03_negitive,"The texts express widespread anxiety, frustrat..."
3,2017_04_negitive,The texts reflect widespread anxiety and negat...
4,2017_05_negitive,The sentences express a strong dislike and fea...
5,2017_06_negitive,The texts discuss various aspects of vaccinati...
6,2017_07_negitive,The text discusses various challenges regardin...
7,2017_08_negitive,The text discusses various opinions and experi...
8,2017_09_negitive,The text reflects a societal discourse around ...
9,2017_10_negitive,The sentences discuss a range of opinions and ...


In [177]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcep7zlGCrW4NK4QGA2G2etVmvdF at 0x254196fd220> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various frustrations and fears related to vaccines, particularly highlighting experiences of pain and anxiety during vaccinations. It mentions the controversy surrounding the dengue vaccine, emphasizing that it should not be administered to individuals who have never had dengue due to potential risks. Concerns about misinformation and vaccine hesitancy are also present, alongside personal anecdotes about the emotional impact of vaccinations on both adults and children.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861803,
  "id": "chatcmpl-AUcep7zlGCrW4NK4QGA2G2etVmvdF",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "

## December

In [178]:
# Get monthly data
df_2017_12_neg = pd.read_csv('../data/local/df_neg_2017-12.csv').reset_index()
df_2017_12_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   959 non-null    int64  
 1   LocationCode            959 non-null    object 
 2   LocationName            959 non-null    object 
 3   StatesCode              959 non-null    object 
 4   ...4                    959 non-null    int64  
 5   id                      959 non-null    float64
 6   text                    959 non-null    object 
 7   label                   959 non-null    object 
 8   in_reply_to_status_id   134 non-null    float64
 9   in_reply_to_user_id     136 non-null    float64
 10  quoted_user_id          34 non-null     float64
 11  quoted_status_id        34 non-null     float64
 12  retweeted_user_id       114 non-null    float64
 13  retweeted_status_id     114 non-null    float64
 14  created_at              959 non-null    ob

### Representative sample based on week and location

In [179]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2017_12_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2017_12_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2017-w48,Acre,1,0.001043
1,2017-w48,Amazonas,2,0.002086
2,2017-w48,Bahia,3,0.003128
3,2017-w48,Ceara,4,0.004171
4,2017-w48,Distrito Federal,6,0.006257
...,...,...,...,...
88,2017-w53,Ceara,1,0.001043
89,2017-w53,Minas Gerais,2,0.002086
90,2017-w53,Parana,1,0.001043
91,2017-w53,Rio Grande Do Sul,1,0.001043


In [180]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2017-w48,Acre,1,0.001043,2
1,2017-w48,Amazonas,2,0.002086,5
2,2017-w48,Bahia,3,0.003128,7
3,2017-w48,Ceara,4,0.004171,10
4,2017-w48,Distrito Federal,6,0.006257,15
...,...,...,...,...,...
88,2017-w53,Ceara,1,0.001043,2
89,2017-w53,Minas Gerais,2,0.002086,5
90,2017-w53,Parana,1,0.001043,2
91,2017-w53,Rio Grande Do Sul,1,0.001043,2


In [181]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2017_12_neg_sample_size = pd.merge(df_2017_12_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [184]:
# Step 6: Perform the stratified sampling
#df_2017_12_neg_sample = df_2017_12_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2017_12_neg_sample = df_2017_12_neg
# Step 7: Check the result
print(f"Sampled {len(df_2017_12_neg_sample)} rows.")


Sampled 959 rows.


In [185]:
text_gpt_2017_12_neg = '\n'.join(df_2017_12_neg_sample['text'].astype(str))

### Prompts and loops

In [186]:
prompt_12 = prompt_raw.replace('TWEETS', text_gpt_2017_12_neg)

In [187]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_12}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [188]:
response.choices[0].message.content

"The sentences express a variety of sentiments regarding vaccinations, particularly the discomfort, pain, and fears associated with them. Many individuals share their experiences of pain after receiving vaccines, particularly for conditions like tetanus and yellow fever. Others express reluctance or anxiety about vaccination, citing negative experiences or side effects. There's also mention of public health campaigns, concerns over the effectiveness and safety of certain vaccines, and a general sense of frustration with the vaccination process."

In [189]:
#summary = []
#dataset = []

In [190]:
dataset.append("2017_12_negitive")

In [191]:
summary.append(response.choices[0].message.content)

In [192]:
df_2017_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2017_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  12 non-null     object
 1   summary  12 non-null     object
dtypes: object(2)
memory usage: 320.0+ bytes


In [193]:
df_2017_neg.tail(14)

,dataset,summary
0,2017_01_negitive,The passages mainly express frustration and fe...
1,2017_02_negitive,The sentences express a common frustration and...
2,2017_03_negitive,"The texts express widespread anxiety, frustrat..."
3,2017_04_negitive,The texts reflect widespread anxiety and negat...
4,2017_05_negitive,The sentences express a strong dislike and fea...
5,2017_06_negitive,The texts discuss various aspects of vaccinati...
6,2017_07_negitive,The text discusses various challenges regardin...
7,2017_08_negitive,The text discusses various opinions and experi...
8,2017_09_negitive,The text reflects a societal discourse around ...
9,2017_10_negitive,The sentences discuss a range of opinions and ...


In [194]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcfmLStEdaNiR0nFzRJn5Q4LyBkF at 0x254196f0db0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences express a variety of sentiments regarding vaccinations, particularly the discomfort, pain, and fears associated with them. Many individuals share their experiences of pain after receiving vaccines, particularly for conditions like tetanus and yellow fever. Others express reluctance or anxiety about vaccination, citing negative experiences or side effects. There's also mention of public health campaigns, concerns over the effectiveness and safety of certain vaccines, and a general sense of frustration with the vaccination process.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731861862,
  "id": "chatcmpl-AUcfmLStEdaNiR0nFzRJn5Q4LyBkF",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  

## All 2017 negative

In [195]:
df_2017_neg.to_csv('../data/summary_2017_negative_tweets.csv')